In [3]:
import arcpy
import traceback

In [4]:


def merge_small_parcels(input_fc, output_fc, merge_field, area_threshold):
    try:
        # 复制输入要素类
        arcpy.CopyFeatures_management(input_fc, output_fc)
        print(f"已创建输出要素类: {output_fc}")

        # 确保存在面积字段
        if "Area" not in [f.name for f in arcpy.ListFields(output_fc)]:
            arcpy.AddField_management(output_fc, "Area", "DOUBLE")
            print("已添加 Area 字段")

        # 计算面积
        arcpy.CalculateField_management(output_fc, "Area", "!shape.area!", "PYTHON3")
        print("已计算面积")

        # 创建图层
        arcpy.MakeFeatureLayer_management(output_fc, "output_layer")

        # 获取小面积图斑
        small_parcels = []
        with arcpy.da.SearchCursor(output_fc, ["OID@", merge_field, "Area", "SHAPE@"]) as cursor:
            for row in cursor:
                oid, field_value, area, shape = row
                if area < area_threshold:
                    small_parcels.append((oid, field_value, area, shape))

        print(f"找到 {len(small_parcels)} 个小于指定阈值的图斑")

        # 处理小面积图斑
        merged_count = 0
        with arcpy.da.Editor(arcpy.Describe(output_fc).path) as editor:
            for small_parcel in small_parcels:
                oid, field_value, area, shape = small_parcel
                
                # 选择相邻图斑
                arcpy.SelectLayerByLocation_management("output_layer", "BOUNDARY_TOUCHES", shape)
                
                # 在选择集中查找最大的相同字段值图斑
                max_area = 0
                max_oid = None
                max_field_value = None
                with arcpy.da.SearchCursor("output_layer", ["OID@", merge_field, "Area"]) as neighbor_cursor:
                    for neighbor in neighbor_cursor:
                        if neighbor[0] != oid:
                            if neighbor[1] == field_value and neighbor[2] > max_area:
                                max_area = neighbor[2]
                                max_oid = neighbor[0]
                                max_field_value = neighbor[1]
                            elif area < 200 and neighbor[2] > max_area:  # 新增：如果面积小于50，考虑所有相邻图斑
                                max_area = neighbor[2]
                                max_oid = neighbor[0]
                                max_field_value = neighbor[1]
                
                if max_oid:
                    try:
                        # 合并几何
                        where_clause = f"OBJECTID IN ({oid}, {max_oid})"
                        with arcpy.da.UpdateCursor(output_fc, ["OID@", "SHAPE@", merge_field], where_clause) as update_cursor:
                            shapes = []
                            for update_row in update_cursor:
                                shapes.append(update_row[1])
                                if update_row[0] == oid:
                                    update_cursor.deleteRow()
                            
                            if len(shapes) == 2:
                                merged_shape = shapes[0].union(shapes[1])
                                update_cursor.reset()
                                for update_row in update_cursor:
                                    if update_row[0] == max_oid:
                                        update_cursor.updateRow([max_oid, merged_shape, max_field_value])
                                        merged_count += 1
                                        break
                    except Exception as e:
                        print(f"处理 OID {oid} 时出错: {str(e)}")
                        print(traceback.format_exc())
                
                # 清除选择
                arcpy.SelectLayerByAttribute_management("output_layer", "CLEAR_SELECTION")

                if merged_count % 100 == 0:
                    print(f"已处理 {merged_count} 个小面积图斑")

        # 重新计算面积
        arcpy.CalculateField_management(output_fc, "Area", "!shape.area!", "PYTHON3")
        print(f"操作完成。合并了 {merged_count} 个小面积图斑到相邻图斑中。")
        print(f"结果保存在新的要素类: {output_fc}")

    except Exception as e:
        print(f"发生错误: {str(e)}")
        print(traceback.format_exc())

In [5]:

input_fc = r'C:\Users\Runker\Desktop\river\temp.gdb\sb_dz_ele_2'
output_fc = r'C:\Users\Runker\Desktop\river\temp.gdb\sb_dz_ele_3'
merge_field = "MZMC"  # 用于合并的字段名
area_threshold = 10000  # 面积阈值，单位与输入数据一致

merge_small_parcels(input_fc, output_fc, merge_field, area_threshold)

已创建输出要素类: C:\Users\Runker\Desktop\river\temp.gdb\sb_dz_ele_3
已计算面积
找到 1060 个小于指定阈值的图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
操作完成。合并了 57 个小面积图斑到相邻图斑中。
结果保存在新的要素类: C:\Users\Runker\Desktop\river\temp.gdb\sb_dz_ele_3
